In [ ]:
import json
from pathlib import Path
import metadata
import numpy as np
import shutil
import datetime as dt
import pytz 

### When to run this notebook

After generating PARGE radiance supercubes through integrated processing, or stacking VNIR and SWIR cubes into full supercubes `[flightline]_VNIR_SWIR_supercube.geo.bsq`, and assembling full-supercubes `[flightline]_VNIR_SWIR_supercube.geo_sca.bsq` (for example using notebook 06). There are a number of ways to accomplish this, among them:

* Manually stacking VNIR and SWIR output using ENVI
* Stacking VNIR and SWIR output programmatically with the `gdal` tools
* Using PARGE's integrated processing to produce a single supercube

The code in the following is quite flexible and needs to be run thoughtfully. At the time of this writing, we presume EITHER integrated processing with cutover at 954 nm to generate 459 bands, or manual stacking in ENVI, using bands 1-170 VNIR and 2-288 SWIR for a total of 457 bands

The code can also be run after ATCOR, for final product generation.

The goal of this code is to produce suitable metadata for use in ATCOR and prepare final packaging metadata.

In [ ]:
def strip_braces(mystr):
    return mystr.replace('{', '').replace('}', '').strip()

In [ ]:
def metadata_to_dict(metadatalist):
    """Take metadata in list-of-lines form, parse into dict"""
    metadict = {}
    for line in metadatalist:
        if line.startswith(' '):
            try:
                metadict[lastkey] = metadict[lastkey] + '\n ' + line.strip()
                continue
            except KeyError:
                continue
        newitem = [item.strip() for item in line.split('=', 1)]
        try:
            metadict[newitem[0]] = metadict[newitem[0]] + ' ' + newitem[1]
        except KeyError:
            try:
                metadict[newitem[0]] = newitem[1]
                lastkey = newitem[0]
            except IndexError:
                metadict[lastkey] = metadict[lastkey] + '\n ' + newitem[0]
    return metadict

def hdrfile_to_dict(fp):
    """Take file path, return dictionary""" 
    with open(fp) as src:
        lines = src.readlines()
        metadata = metadata_to_dict(lines[1:])
    return metadata


In [ ]:
projdir = "Z://fihyper//cwaigl//20200710_CPC"
prefix = "20200710-CPC"

### Prototyping

In [ ]:
lineno = '14' 
navdir = Path(f"{projdir}\\{prefix}_{lineno}\\NAV")
atcordir = Path(f"{projdir}\\{prefix}_{lineno}\\ATCOR-1130")
brefcordir = Path(f"{projdir}\\02_intermediate\\brefcor")
pargedir = Path(f"{projdir}\\{prefix}_{lineno}\\ATCOR")
outputdir = Path(f"{projdir}\\02_intermediate\\working")

In [ ]:
extra = '_rad' 

In [ ]:
targetfile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr'
atcorfile = f'{prefix}_{lineno}_VNIR_SWIR_supercube_masked_geo_atm.hdr'
pargefile = f'{prefix}_{lineno}_VNIR_SWIR_supercube_masked_geo.hdr'
flightdatafile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_flightdata.txt'

In [ ]:
targetfile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_atm_bcor.hdr'
atcorfile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_atm.hdr'
pargefile = f'{prefix}_{lineno}_VNIR_SWIR{extra}_geo.hdr'
flightdatafile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_flightdata.txt'

In [ ]:
pargemeta = hdrfile_to_dict(pargedir / pargefile)
atcormeta = hdrfile_to_dict(atcordir / atcorfile)
targetmeta = hdrfile_to_dict(brefcordir / targetfile)
flightdata = json.load(open(navdir / flightdatafile))

In [ ]:
flightdata

{'flightlinename': '20200710-CPC_14',
 'origts_utc': '2020-07-11T01:33:33.0Z',
 'linets_utc': '2020-07-11T01:38:09.0Z',
 'linets_akdt': '2020-07-10 17:38:09',
 'heading_avg': 182.67,
 'roll_avg': -1.27,
 'roll_std': 1.84,
 'pitch_avg': -7.25,
 'pitch_std': 1.09,
 'elevation_m_amsl': 2335.72,
 'latitude': 65.16,
 'longitude': -147.51,
 'sun_azimuth': 247.51,
 'sun_zenith': 56.0}

In [ ]:
targetmeta

{'description': '{\n 20200710-CPC_14_VNIR_SWIR_rad_geo_atm_bcor_crop.bsq}',
 'samples': '1128',
 'lines': '6503',
 'bands': '457',
 'header offset': '0',
 'file type': 'ENVI Standard',
 'data type': '2',
 'interleave': 'bsq',
 'byte order': '0',
 'map info': '{UTM, 1, 1, 475388.5, 7230445.5, 1, 1, 6, North,WGS-84}',
 'coordinate system string': '{PROJCS["WGS_1984_UTM_Zone_6N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-147.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]}',
 'band names': '{\n band 1 refl [%*100] (410.36),\n band 2 refl [%*100] (413.53),\n band 3 refl [%*100] (416.70),\n band 4 refl [%*100] (419.87),\n band 5 refl [%*100] (423.04),\n band 6 refl [%*100] (426.21),\n band 7 refl [%*100] (429.38),\n b

In [ ]:
atcormeta

{'samples': '1901',
 'lines': '10151',
 'bands': '457',
 'header offset': '0',
 'data type': '2',
 'byte order': '0',
 'interleave': 'BSQ',
 'file type': 'ENVI Standard',
 'default bands': '{290, 140, 20}',
 'description': '{ATCOR4 : 11\\07\\20  solar zen=56.0,  solar azi=247.5,  sensorid=HySpex_457_FoV2_Husky,\n fcref=100.0,  fctem=100.0}',
 'x start': '1.00000',
 'y start': '1.00000',
 'pixel size': '{       1.0000000,        1.0000000}',
 'map info': '{UTM,        1,        1,        474899.50,        7232100.5,  1.0000000000e+000,  1.0000000000e+000,  6,  North,  WGS-84,  units=Meters}',
 'z plot titles': '{ wavelength [nm], reflectance [%*100] }',
 'band names': '{band   1 refl [%*100], band   2 refl [%*100], band   3 refl [%*100],\n band   4 refl [%*100], band   5 refl [%*100], band   6 refl [%*100], band   7 refl [%*100],\n band   8 refl [%*100], band   9 refl [%*100], band  10 refl [%*100], band  11 refl [%*100],\n band  12 refl [%*100], band  13 refl [%*100], band  14 refl [%*

In [ ]:
pargemeta

{'description': '{Frameperiod = 12000 Integration time = 8300 Binning = 1 Number of frames = 17299 Aperture size =   0.008000 dw = 1 EQ = 0 FOVexp = 5 Lens = 5 NumberOfAvg = 1 CalibAvailable = 0 Number of background = 200 Pixelsize x =   0.000320 Pixelsize y =   0.000640 ID = VNIR_1800_SN00812 Comment =  Serialnumber = 812 Scanningmode = Airborne; PARGE Geocoded  20200710-CPC_14_VNIR_1800_SN00812_FOVx2_raw_rad_bsq_float32 ENVI-File}',
 'samples': '1901',
 'lines': '10151',
 'bands': '457',
 'header offset': '0',
 'file type': 'ENVI Standard',
 'data type': '4',
 'interleave': 'bsq',
 'sensor type': 'HYSPEX',
 'byte order': '0',
 'map info': '{UTM, 1.000, 1.000, 474899.500, 7232100.500, 1.0000000000e+000, 1.0000000000e+000, 6, North, WGS-84, units=Meters}',
 'coordinate system string': '{PROJCS["WGS_1984_UTM_Zone_6N",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mer

In [ ]:
replacements = {
    'acquisition time': flightdata['linets_utc'],
    'reflectance scale factor': atcormeta['reflectance scale factor'],
    'default bands': '{289.000000, 141.000000, 20.000000}' if targetmeta['bands'] == 457 else '{291.000000, 141.000000, 20.000000}',
    'description':  ( 
        f"{{Spectral reflectance (%*100) for HySpex flightline {flightdata['flightlinename']}. "
        f"central lat: {flightdata['latitude']}, lon: {flightdata['longitude']}, flight elev: {flightdata['elevation_m_amsl']} m above MSL. "
        f"PARGE (VNIR): {strip_braces(pargemeta['description'])} "
        f"ATCOR:  {strip_braces(atcormeta['description'])} }}" ),
    'band names': atcormeta['band names'].replace(' refl [%*100]', ''),
    'wavelength': atcormeta['wavelength'],
    'wavelength units': 'nm',
    'fwhm': atcormeta['fwhm'],
    'z plot titles': atcormeta['z plot titles'],
    'z plot range': '{0, 10000}',
    'sun azimuth': flightdata['sun_azimuth'],
    'sun elevation': 90.0 - flightdata['sun_zenith'],
}


In [ ]:
replacements

{'acquisition time': '2020-07-11T01:38:09.0Z',
 'reflectance scale factor': '10000',
 'default bands': '{291.000000, 141.000000, 20.000000}',
 'description': '{Spectral reflectance (%*100) for HySpex flightline 20200710-CPC_14. central lat: 65.16, lon: -147.51, flight elev: 2335.72 m above MSL. PARGE (VNIR): Frameperiod = 12000 Integration time = 8300 Binning = 1 Number of frames = 17299 Aperture size =   0.008000 dw = 1 EQ = 0 FOVexp = 5 Lens = 5 NumberOfAvg = 1 CalibAvailable = 0 Number of background = 200 Pixelsize x =   0.000320 Pixelsize y =   0.000640 ID = VNIR_1800_SN00812 Comment =  Serialnumber = 812 Scanningmode = Airborne; PARGE Geocoded  20200710-CPC_14_VNIR_1800_SN00812_FOVx2_raw_rad_bsq_float32 ENVI-File ATCOR:  ATCOR4 : 11\\07\\20  solar zen=56.0,  solar azi=247.5,  sensorid=HySpex_457_FoV2_Husky,\n fcref=100.0,  fctem=100.0 }',
 'band names': '{band   1, band   2, band   3,\n band   4, band   5, band   6, band   7,\n band   8, band   9, band  10, band  11,\n band  12, b

In [ ]:
targetmeta.update(replacements)

In [ ]:
targetmeta

{'description': '{Spectral reflectance (%*100) for HySpex flightline 20200710-CPC_14. central lat: 65.16, lon: -147.51, flight elev: 2335.72 m above MSL. PARGE (VNIR): Frameperiod = 12000 Integration time = 8300 Binning = 1 Number of frames = 17299 Aperture size =   0.008000 dw = 1 EQ = 0 FOVexp = 5 Lens = 5 NumberOfAvg = 1 CalibAvailable = 0 Number of background = 200 Pixelsize x =   0.000320 Pixelsize y =   0.000640 ID = VNIR_1800_SN00812 Comment =  Serialnumber = 812 Scanningmode = Airborne; PARGE Geocoded  20200710-CPC_14_VNIR_1800_SN00812_FOVx2_raw_rad_bsq_float32 ENVI-File ATCOR:  ATCOR4 : 11\\07\\20  solar zen=56.0,  solar azi=247.5,  sensorid=HySpex_457_FoV2_Husky,\n fcref=100.0,  fctem=100.0 }',
 'samples': '1128',
 'lines': '6503',
 'bands': '457',
 'header offset': '0',
 'file type': 'ENVI Standard',
 'data type': '2',
 'interleave': 'bsq',
 'byte order': '0',
 'map info': '{UTM, 1, 1, 475388.5, 7230445.5, 1, 1, 6, North,WGS-84}',
 'coordinate system string': '{PROJCS["WGS_1

---

Write new header file

In [ ]:
shutil.copyfile(brefcordir / targetfile, brefcordir / f"{targetfile}.bak")

WindowsPath('Z:/fihyper/cwaigl/20200710_CPC/02_intermediate/brefcor/20200710-CPC_14_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr.bak')

In [ ]:
with open(brefcordir / targetfile, "w") as dst:
    dst.write("ENVI\n")
    for key in targetmeta:
        dst.write(f"{key} = {targetmeta[key]}\n")

In [ ]:
brefcordir / targetfile

WindowsPath('Z:/fihyper/cwaigl/20200710_CPC/02_intermediate/brefcor/20200710-CPC_14_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr')

---

### Now loop through all

In [ ]:
projdir = "Z:\\fihyper\\cwaigl\\20210803_BC"
prefix = "20210803-BC"
crop = True

brefcordir = Path(f"{projdir}\\02_intermediate\\brefcor")
outputdir = Path(f"{projdir}\\02_intermediate\\working")

extra = '_rad'
if crop:
    cropextra = "_crop"
    textextra = " cropped"
else:
    cropextra = ""
    textextra = ""

startline = 1
endline = 7

linenos = [str(ii).zfill(2) for ii in range(startline, endline+1)]

In [ ]:
(brefcordir / f"{targetfile}.bak").exists()

False

In [ ]:
dt.datetime.now().strftime("%Y%m%d_%H%M%S")

'20220417_000755'

In [ ]:
for lineno in linenos:
    navdir = Path(f"{projdir}\\{prefix}_{lineno}\\NAV")
    pargedir = Path(f"{projdir}\\{prefix}_{lineno}\\PARGE")
    print(pargedir)
    atcordir = Path(f"{projdir}\\{prefix}_{lineno}\\ATCOR")
    targetfile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_atm_bcor{cropextra}.hdr'
    atcorfile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_atm.hdr'
    pargefile = f'{prefix}_{lineno}_VNIR_SWIR{extra}_geo.hdr'
    flightdatafile = f'{prefix}_{lineno}_VNIR_SWIR_rad_geo_flightdata.txt'
    pargemeta = hdrfile_to_dict(pargedir / pargefile)
    atcormeta = hdrfile_to_dict(atcordir / atcorfile)
    targetmeta = hdrfile_to_dict(brefcordir / targetfile)
    flightdata = json.load(open(navdir / flightdatafile))
    replacements = {
        'acquisition time': flightdata['linets_utc'],
        'reflectance scale factor': atcormeta['reflectance scale factor'],
        'default bands': '{289.000000, 141.000000, 20.000000}' if targetmeta['bands'] == 457 else '{291.000000, 141.000000, 20.000000}',
        'description':  ( 
            f"{{Spectral reflectance (%*100) for{textextra} HySpex flightline {flightdata['flightlinename']}. "
            f"central lat: {flightdata['latitude']}, lon: {flightdata['longitude']}, flight elev: {flightdata['elevation_m_amsl']} m above MSL. "
            f"PARGE (VNIR): {strip_braces(pargemeta['description'])} "
            f"ATCOR:  {strip_braces(atcormeta['description'])} }}" ),
        'band names': atcormeta['band names'].replace(' refl [%*100]', ''),
        'wavelength': atcormeta['wavelength'],
        'wavelength units': 'nm',
        'fwhm': atcormeta['fwhm'],
        'z plot titles': atcormeta['z plot titles'],
        'z plot range': '{0, 10000}',
        'sun azimuth': flightdata['sun_azimuth'],
        'sun elevation': 90.0 - flightdata['sun_zenith'],
    }
    print(f"updating {targetfile}")
    targetmeta.update(replacements)
    if (brefcordir / f"{targetfile}.bak").exists():
        timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        shutil.copyfile(brefcordir / targetfile, brefcordir / f"{targetfile}.{timestamp}.bak")
    else:
        shutil.copyfile(brefcordir / targetfile, brefcordir / f"{targetfile}.bak")
    with open(brefcordir / targetfile, "w") as dst:
        dst.write("ENVI\n")
        for key in targetmeta:
            dst.write(f"{key} = {targetmeta[key]}\n")

Z:\fihyper\cwaigl\20210803_BC\20210803-BC_01\PARGE
updating 20210803-BC_01_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_02\PARGE
updating 20210803-BC_02_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_03\PARGE
updating 20210803-BC_03_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_04\PARGE
updating 20210803-BC_04_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_05\PARGE
updating 20210803-BC_05_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_06\PARGE
updating 20210803-BC_06_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
Z:\fihyper\cwaigl\20210803_BC\20210803-BC_07\PARGE
updating 20210803-BC_07_VNIR_SWIR_rad_geo_atm_bcor_crop.hdr
